In [1]:
import os
import numpy as np
import pandas as pd

from sys import path
path.append("../analysis/utils/")

from utils import get_datasets

In [8]:
DATASETS = ["webkb", "20ng", "reut", "acm"]

CLFS = ["kpr", "ktr", "lpr", "ltr", "sfr", "stmk", "xfr", "xpr", "xtr", "kfr", "ktmk", "lfr", "ltmk", "spr", "str", "xlnet_softmax", "xtmk", "rep_bert"]

DATA_SOURCE = "/home/welton/data"

TR = 3
MIN_MAX = 1 # 0 MIN, 1 MAX.

def set_label(MIN_MAX):
    # If min
    if MIN_MAX == 0:
        return "MIN"
    return "MAX"

def make_comp(MIN_MAX, TR, v1):

    # If MIN
    if MIN_MAX == 0:
        return v1 > TR
    return v1 < TR

ORACLE_DIR = f"{DATA_SOURCE}/oracle/upper_test_HITS-{set_label(MIN_MAX)}-{TR}"

In [5]:
pd_datasets = get_datasets(DATASETS, path=f"{DATA_SOURCE}/pd_datasets/__dset__.csv", sep=';')

In [6]:
# For each dataset.
for dset in DATASETS:
    df = pd_datasets[dset]
    try:
        preds = df.drop(columns=["label", "fold_id", "rep_bert_bkp", "bert"])
    except:
        preds = df.drop(columns=["label", "fold_id", "bert"])
    # For each classifier.
    for clf in CLFS:
        # For each fold.
        for fold in np.arange(10):

            test_load = np.load(f"/home/welton/data/clfs_output/split_10/{dset}/10_folds/{clf}/{fold}/test.npz")
            test_preds = test_load["X_test"].argmax(axis=1)
            test_labels = np.load(f"/home/welton/data/datasets/labels/split_10/{dset}/{fold}/test.npy")

            output_dir = f"{ORACLE_DIR}/{dset}/{clf}/{fold}/"
            os.makedirs(output_dir, exist_ok=True)

            # Making test upper bound.
            test_upper = np.zeros(test_preds.shape[0])

            # Set Hit/True (value 1) on documents right documentos or documents where
            # the agreement size is shorter than AGSIZE.
            c1 = test_preds == test_labels
            c2 = df[df.fold_id == fold].hit_counts > TR
            test_upper[c1 | c2] = 1

            output = f"{output_dir}/test"
            # Save the upper_ground.
            np.savez(output, y=test_upper)

            # Generating upper test.
            output_dir = f"{ORACLE_DIR}/{dset}/{clf}/{fold}/"
            os.makedirs(output_dir, exist_ok=True)
            output = f"{output_dir}/train"
            size = np.load(f"/home/welton/data/datasets/labels/split_10/{dset}/{fold}/train.npy").shape[0]
            fake_train = np.zeros(size) + 1
            np.savez(output, y=fake_train)
            output = f"{output_dir}/test"
            np.savez(output, y=test_upper)
            

In [45]:
np.array([True, False, False]) | np.array([True, False, True])

array([ True, False,  True])

In [46]:
df = pd_datasets["webkb"]
for fold in np.arange(10):
    y = np.load(f"/home/welton/data/datasets/labels/split_10/webkb/{fold}/test.npy")
    print(f"FOLD - {fold} -- {np.unique(df[df.fold_id == fold].label.values == y)}")


FOLD - 0 -- [ True]
FOLD - 1 -- [ True]
FOLD - 2 -- [ True]
FOLD - 3 -- [ True]
FOLD - 4 -- [ True]
FOLD - 5 -- [ True]
FOLD - 6 -- [ True]
FOLD - 7 -- [ True]
FOLD - 8 -- [ True]
FOLD - 9 -- [ True]


In [50]:
np.unique(np.load(f"/home/welton/data/oracle/upper_test_MIN-{AGSIZE}/webkb/kfr/0/test.npz")['y'], return_counts=True)

(array([0., 1.]), array([116, 707]))

In [51]:
np.unique(np.load(f"/home/welton/data/oracle/upper_test_MIN-{AGSIZE}/webkb/kfr/0/train.npz")['y'])

array([1.])

In [52]:
np.load(f"/home/welton/data/oracle/upper_test_MIN-{AGSIZE}/webkb/rep_bert/0/test.npz")['y']

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 0., 1.